In [1]:
import pandas as pd
import numpy as np
import pydeck as pdk
from os import environ, getenv, path
from typing import TypedDict, Dict, Union, List
from simulator_funcs import simulate, load_ship, get_routing_table_from_mmsi, plot_df, get_routing_and_emissions, emissions_multiple_vessels

import sys
sys.path.append('/home/jovyan/odp-python-sdk')

from odp.database_functions.lookup_ship_data import fetch_ship_data
from odp.database_functions.database import get_connection_pool, get_engine
import typing

NameError: name 'CapacityBin' is not defined

## Color code explanation
<p style="color:#FE774C; font-size:12pt;"> Red means that values <i>have</i> to be inserted.</p> 
<p style="color:#9D59F4; font-size:12pt"> Purple means that values <i>can</i> be inserted if other that the default values are needed.</p> 


## Database connection
Getting a connection pool to connect to the database

In [ ]:
#tcp = get_connection_pool()

<p> </p> 
 <p style="color:#FE774C; font-size:20pt; font-weight:bold"> Insert all the mmsis for the vessels interest in here </p> 
<p> For example like the list under with two vessels (change the mmsis and add more of them if needed). </p> 

In [ ]:
mmsis = [212552000, 319009700]

<p> </p> 
 <p style="color:#FE774C; font-size:20pt; font-weight:bold"> Insert the coordinates of the ports in the simulated paths </p> 
<p> For example like the list under where the path will go from Oslo to Kiel to Amsterdam. </p> 

In [ ]:
coordinates = [[10.736223,59.904479], [10.143301,54.320150],[4.8079,52.412]]

# Specifying properties needed for routing

In the following cells we are specifying speed and draught for the vessels as well as graphing, time resolution, how detailed the routing should be and how much other vessel data should be weighed for the routing. 

<p style="color:#9D59F4; font-size:15pt; font-weight:bold"> Make a list for the speeds to simulate with for each vessel  </p> 
<p> If the speed is set to None, the maximum speed for the vessel is chosen. </p> 

In [ ]:
speeds = [None]*len(mmsis)
#Or:
#speeds = [10, 10] #Here, the speed for both vessels is set to 10 knots
speeds

<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> List of draught for the different vessels </p> 
<p> If they are set to None, the draught will be found in the data about the vessel particulars. </p> 

In [ ]:
draughts = [None]*len(mmsis)
#Or:
#draughts = [7, 9] #here the draught for the first ship is 7m and for the second ship it is 9 m
draughts

<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> Choose grapfing types for the vessels </p> 
<p> This is also found from the vessel particulars, but in the incidences where it cannot be found, it needs to be specified. Possible graph types are passenger, fishing, cargo, tanker and tug. </p> 

In [ ]:
graphs = [None] * len(mmsis)
#Or:
#graphs = ['cargo', 'tug'] 
graphs

### Make a dataframe of the mmsis, speeds, draughts and graphs
This is used as input in the function finding routes and emissions.

In [ ]:
zipped_lists = list(zip(speeds, draughts, graphs))
df_input_properties = pd.DataFrame(zipped_lists, index = mmsis, columns = ["speeds","draughts","graphs"])

In [ ]:
df_input_properties

<p> </p>
<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> Choose the time resolution for the graphing </p> 
<p> The default value is every 60th minute. Choose a value between 5 and 240. </p> 

In [ ]:
time_resolution = 60

<p> </p>
<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> Decide how fine/coarse the routing should be </p> 

 * 0 = Coarse (fast)
 * 1 = Coarse + refined
 * 2 = Fine (slow)

In [ ]:
detailed_routing = 0

<p> </p>
<p style="color:#9D59F4; font-size:14pt; font-weight:bold"> How much should popular routes be weighted in the routing? </p> 
<p> Choose a value between 0 and 0.95. Default is 0.3. </p> 

In [ ]:
cost_density = 0.3

## Finding paths and emissions for each vessel

Using a premade function called emissions_multiple_vessels that takes the coordinates chosen and the df_input_properties as arguments. OBS, this might take a while. 

In [ ]:
%%time
df_summed_emissions, list_of_df_emissions, list_of_paths = emissions_multiple_vessels(coordinates,df_input_properties, 1000, 1000, time_resolution, detailed_routing, cost_density)


### Summed emissions for all vessels compared

In [ ]:
df_summed_emissions

## Show the paths
This one is for the vessel that uses the least amount of CO2

In [ ]:
min_co2_mmsi = int(df_summed_emissions.index[0])
min_co2_mmsi

In [ ]:
list_of_paths[mmsis.index(min_co2_mmsi)]

In [ ]:
#max_co2_mmsi = int(df_summed_emissions.index[1])
#list_of_paths[mmsis.index(max_co2_mmsi)]